In [25]:
import yfinance as yf
import pandas as pd

symbols = ["BREN.JK", "TPIA.JK", "PTRO.JK", "CUAN.JK"]
all_data = []

for symbol in symbols:
    df = yf.Ticker(symbol).history(
        start="2025-01-01",
        end="2026-01-31",
        auto_adjust=False
    )

    if df.empty:
        continue

    df = df.reset_index()
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]
    df["symbol"] = symbol

    all_data.append(df)

final_df = pd.concat(all_data, ignore_index=True)

final_df.columns = [c.lower() for c in final_df.columns]

# 🔴 INI YANG KURANG TADI
final_df = final_df.sort_values(
    by=["date", "symbol"]
).reset_index(drop=True)

final_df.head(10)


,date,open,high,low,close,volume,symbol
0,2025-01-02 00:00:00+07:00,9275.0,9825.0,9275.0,9475.0,31228000,BREN.JK
1,2025-01-02 00:00:00+07:00,1117.5,1220.0,1117.5,1180.0,230169000,CUAN.JK
2,2025-01-02 00:00:00+07:00,2795.0,2832.5,2625.0,2745.0,75120000,PTRO.JK
3,2025-01-02 00:00:00+07:00,7475.0,7600.0,7375.0,7500.0,6072100,TPIA.JK
4,2025-01-03 00:00:00+07:00,9525.0,9750.0,9450.0,9675.0,16136000,BREN.JK
5,2025-01-03 00:00:00+07:00,1205.0,1215.0,1162.5,1200.0,117490000,CUAN.JK
6,2025-01-03 00:00:00+07:00,2820.0,2980.0,2790.0,2880.0,126582100,PTRO.JK
7,2025-01-03 00:00:00+07:00,7525.0,7675.0,7175.0,7175.0,9967800,TPIA.JK
8,2025-01-06 00:00:00+07:00,9700.0,9925.0,9700.0,9800.0,19534300,BREN.JK
9,2025-01-06 00:00:00+07:00,1215.0,1350.0,1215.0,1270.0,239027000,CUAN.JK


In [44]:
# pastikan urut
final_df = final_df.sort_values(
    by=["symbol", "date"]
).reset_index(drop=True)

# hitung daily return per saham
final_df["daily_return"] = (
    final_df
    .groupby("symbol")["close"]
    .pct_change()
)

final_df.tail(3)


,date,open,high,low,close,volume,symbol,daily_return,cumulative_return,equity_curve
1021,2026-01-28 00:00:00+07:00,6700.0,6825.0,6075.0,6600.0,14906300,TPIA.JK,-0.057143,-0.12,880000.0
1022,2026-01-29 00:00:00+07:00,6350.0,6500.0,5650.0,6300.0,21595600,TPIA.JK,-0.045455,-0.16,840000.0
1023,2026-01-30 00:00:00+07:00,6300.0,6450.0,6200.0,6450.0,10070200,TPIA.JK,0.023810,-0.14,860000.0


In [43]:
# kedua kolom dihitung dari awal

initial_capital = 1_000_000  # 1 juta rupiah

# pastikan urut
final_df = final_df.sort_values(
    by=["symbol", "date"]
).reset_index(drop=True)

# cumulative return (tetap sama)
final_df["cumulative_return"] = (
    (1 + final_df["daily_return"])
    .groupby(final_df["symbol"])
    .cumprod()
    - 1
)

# equity curve dalam rupiah
final_df["equity_curve"] = (
    initial_capital * (1 + final_df["cumulative_return"])
)

final_df.head(3)


,date,open,high,low,close,volume,symbol,daily_return,cumulative_return,equity_curve
0,2025-01-02 00:00:00+07:00,9275.0,9825.0,9275.0,9475.0,31228000,BREN.JK,NaN,NaN,NaN
1,2025-01-03 00:00:00+07:00,9525.0,9750.0,9450.0,9675.0,16136000,BREN.JK,0.021108,0.021108,1.021108e+06
2,2025-01-06 00:00:00+07:00,9700.0,9925.0,9700.0,9800.0,19534300,BREN.JK,0.012920,0.034301,1.034301e+06


In [42]:
final_df["equity_curve"] = final_df["equity_curve"].round(0)
final_df.tail(4)

,date,open,high,low,close,volume,symbol,daily_return,cumulative_return,equity_curve
1020,2026-01-27 00:00:00+07:00,6750.0,7000.0,6650.0,7000.0,6585400,TPIA.JK,0.029412,-0.066667,933333.0
1021,2026-01-28 00:00:00+07:00,6700.0,6825.0,6075.0,6600.0,14906300,TPIA.JK,-0.057143,-0.120000,880000.0
1022,2026-01-29 00:00:00+07:00,6350.0,6500.0,5650.0,6300.0,21595600,TPIA.JK,-0.045455,-0.160000,840000.0
1023,2026-01-30 00:00:00+07:00,6300.0,6450.0,6200.0,6450.0,10070200,TPIA.JK,0.023810,-0.140000,860000.0


In [84]:
# 1️⃣ date → YYYY-MM-DD
final_df["date"] = pd.to_datetime(final_df["date"]).dt.date

# 2️⃣ harga → integer (tanpa .0)
price_cols = ["open", "high", "low", "close"]
final_df[price_cols] = final_df[price_cols].round(0).astype(int)

# 3️⃣ symbol → hilangkan .JK
final_df["symbol"] = final_df["symbol"].str.replace(".JK", "", regex=False)

# 4️⃣ cumulative_return → NaN jadi 0
final_df["cumulative_return"] = final_df["cumulative_return"].fillna(0)

# 5️⃣ equity_curve → NaN jadi modal awal, lalu int
final_df["equity_curve"] = (
    final_df["equity_curve"]
    .fillna(1_000_000)
    .round(0)
    .astype(int)
)

final_df.head(200)


,date,open,high,low,close,volume,symbol,daily_return,cumulative_return,equity_curve,daily_return_pct,cumulative_return_pct
0,2025-01-02,9275,9825,9275,9475,31228000,BREN,NaN,0.000000,1000000,NaN,0.00
1,2025-01-03,9525,9750,9450,9675,16136000,BREN,0.021108,0.021108,1021108,2.11,2.11
2,2025-01-06,9700,9925,9700,9800,19534300,BREN,0.012920,0.034301,1034301,1.29,3.43
3,2025-01-07,9850,10550,9750,10550,50713700,BREN,0.076531,0.113456,1113456,7.65,11.35
4,2025-01-08,10600,10650,10200,10475,35618900,BREN,-0.007109,0.105541,1105541,-0.71,10.55
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2025-10-31,8700,8850,8650,8675,11425900,BREN,0.005797,-0.084433,915567,0.58,-8.44
196,2025-11-03,8800,9400,8775,9275,48431600,BREN,0.069164,-0.021108,978892,6.92,-2.11
197,2025-11-04,9400,9750,9175,9225,55631300,BREN,-0.005391,-0.026385,973615,-0.54,-2.64
198,2025-11-05,9200,9750,9125,9750,53216800,BREN,0.056911,0.029024,1029024,5.69,2.90


In [83]:
final_df["daily_return_pct"] = final_df["daily_return"] * 100
final_df["cumulative_return_pct"] = final_df["cumulative_return"] * 100

# biar rapi 2 desimal
final_df["daily_return_pct"] = final_df["daily_return_pct"].round(2)
final_df["cumulative_return_pct"] = final_df["cumulative_return_pct"].round(2)

final_df.head()


,date,open,high,low,close,volume,symbol,daily_return,cumulative_return,equity_curve,daily_return_pct,cumulative_return_pct
0,2025-01-02,9275,9825,9275,9475,31228000,BREN,NaN,0.000000,1000000,NaN,0.00
1,2025-01-03,9525,9750,9450,9675,16136000,BREN,0.021108,0.021108,1021108,2.11,2.11
2,2025-01-06,9700,9925,9700,9800,19534300,BREN,0.012920,0.034301,1034301,1.29,3.43
3,2025-01-07,9850,10550,9750,10550,50713700,BREN,0.076531,0.113456,1113456,7.65,11.35
4,2025-01-08,10600,10650,10200,10475,35618900,BREN,-0.007109,0.105541,1105541,-0.71,10.55


In [85]:
final_df.to_csv(
    "fact_prices_clean.csv",
    index=False,
    encoding="utf-8"
)
